In [126]:
import numpy as np
import os.path
import math

import pandas as pd
from pandas.core.frame import DataFrame

# classificator
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


In [127]:
TARGETS = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [201]:
def parserTypes(dataPd: DataFrame):
    #print(dataPd.head(4))
    size = len(dataPd)
    for i in range(size):
        if dataPd.loc[i,'Sex'] == 'female':
            dataPd.loc[i,'Sex'] = 0
        if dataPd.loc[i,'Sex'] == 'male':
            dataPd.loc[i,'Sex'] = 1

        if dataPd.loc[i,'Embarked'] == 'S':
            dataPd.loc[i,'Embarked'] = 0
        if dataPd.loc[i,'Embarked'] == 'C':
            dataPd.loc[i,'Embarked'] = 1
        if dataPd.loc[i,'Embarked'] != 'C' and dataPd.loc[i,'Embarked'] != 'S':
            dataPd.loc[i,'Embarked'] = 2

        if math.isnan(dataPd.loc[i, 'Age']):
            dataPd.loc[i, 'Age'] = 0
        
        if math.isnan(dataPd.loc[i, 'Fare']):
            dataPd.loc[i, 'Fare'] = 0
    return dataPd
        
        
def loadFile():
    path = os.path.abspath('')
    dataRawPd = pd.read_csv(f'{path}/train.csv')
    survived = dataRawPd['Survived']
    dataPd = dataRawPd[TARGETS]
    return dataPd, survived

def loadTestFile():
    path = os.path.abspath('')
    dataRawPd = pd.read_csv(f'{path}/test.csv')
    results = pd.read_csv(f'{path}/gender_submission.csv')
    dataPd = dataRawPd[TARGETS]
    return dataPd, results

In [244]:
# Train model
rowDataPd, survivedPd = loadFile()
dataPd = parserTypes(rowDataPd)
# dataPd = dataPd.iloc[:, 1:] Remove colums and rows depends on
print('dataPd', dataPd.shape)
print('dataPd', dataPd.head(2))
survived = survivedPd.to_numpy()
data = dataPd.to_numpy()

# Remove column passegerId
data = np.delete(data, 0, 1)

# pd.plotting.scatter_matrix(rowDataPd, c=survived, figsize=(15,15), market='o')
n_neighbors = 5
knn = KNeighborsClassifier(n_neighbors, weights='distance')
knn.fit(data, survived)

log = LogisticRegression(random_state=0, max_iter=1000).fit(data, survived)
# knn.predict()



dataPd (891, 8)
dataPd    PassengerId  Pclass Sex   Age  SibSp  Parch     Fare Embarked
0            1       3   1  22.0      1      0   7.2500        2
1            2       1   0  38.0      1      0  71.2833        2


In [245]:
# load test
rowDataTestPd, resultPd = loadTestFile()
dataTestPd = parserTypes(rowDataTestPd)
#print(dataTestPd["Age"].describe())
# print(dataTestPd["Age"])
dataTest = dataTestPd.to_numpy()
result = resultPd.to_numpy()

# Remove column passegerId and index for results
dataTest = np.delete(dataTest, 0, 1)
result = np.delete(result, 0, 1)

# Validate NaN
for item in dataTest:
    # print(item[5])
    if math.isnan(item[5]):
        print(item)

predict = knn.predict(dataTest)
predictLog = log.predict(dataTest)
predictLog.shape

(418,)

In [246]:
# Validate predictions
np.mean(predictLog == result)

0.5332753371030883